# Training Fetchland

## Setup

We need system utilities to load files, and make system calls:

In [1]:
import os
import sys
import subprocess

[numby](http://www.numpy.org/) - scientific computing package for Python

In [2]:
import numpy

[fontconfig](https://pypi.python.org/pypi/Python-fontconfig/0.5.0): Fontconfig is a library for configuring and customizing font access.  This is the python wrapper around the [Fontconfig](https://www.freedesktop.org/wiki/Software/fontconfig/) library.



In [3]:
import fontconfig

This is how we are making system calls (if needed):

In [4]:
from subprocess import call

[functools](https://docs.python.org/2/library/functools.html) is useful to make decorators:

In [5]:
import functools
from functools import partial

[fonttools](https://github.com/fonttools/fonttools): A library to manipulate font files from Python

In [6]:
from fontTools.ttLib import TTFont

[Pillow](http://pillow.readthedocs.io/en/4.2.x/index.html): Image processing
* `Image` - create, update, read, save, convert Image files
* `ImageDraw` - used to generate text
* `ImageFont` - loads our fonts so we can use them to generte text on our images

In [7]:
from PIL import Image, ImageDraw, ImageFont, ImageEnhance, ImageFilter
filter_dict = {}

In [8]:
from imageprocessing import FontProps, ImageProps
from imageprocessing.image_generator import ImageGenerator

ImportError: No module named enhancement_props

## Validation

### Error Handling

In [ ]:
def ordinal(num):
    '''
    Returns the ordinal number of a given integer, as a string.
    eg. 1 -> 1st, 2 -> 2nd, 3 -> 3rd, etc.
    '''
    if 10 <= num % 100 < 20:
        return '{0}th'.format(num)
    else:
        ord = {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(num % 10, 'th')
        return '{0}{1}'.format(num, ord)

In [ ]:
class ArgumentValidationError(ValueError):
    '''
    Raised when the type of an argument to a function is not what it should be.
    '''
    def __init__(self, arg_num, func_name, accepted_arg_type):
        self.error = 'The {0} argument of {1}() is not a {2}'.format(arg_num,
                                                                     func_name,
                                                                     accepted_arg_type)
 
    def __str__(self):
        return self.error

class InvalidArgumentNumberError(ValueError):
    '''
    Raised when the number of arguments supplied to a function is incorrect.
    Note that this check is only performed from the number of arguments
    specified in the validate_accept() decorator. If the validate_accept()
    call is incorrect, it is possible to have a valid function where this
    will report a false validation.
    '''
    def __init__(self, func_name):
        self.error = 'Invalid number of arguments for {0}()'.format(func_name)
 
    def __str__(self):
        return self.error

class InvalidReturnType(ValueError):
    '''
    As the name implies, the return value is the wrong type.
    '''
    def __init__(self, return_type, func_name):
        self.error = 'Invalid return type {0} for {1}()'.format(return_type,
                                                                func_name)
 
    def __str__(self):
        return self.error


### Decorators

In [ ]:
def accepts(*accepted_arg_types):
    '''
    A decorator to validate the parameter types of a given function.
    It is passed a tuple of types. eg. (<type 'tuple'>, <type 'int'>)
 
    Note: It doesn't do a deep check, for example checking through a
          tuple of types. The argument passed must only be types.
    '''
 
    def accept_decorator(validate_function):
        # Check if the number of arguments to the validator
        # function is the same as the arguments provided
        # to the actual function to validate. We don't need
        # to check if the function to validate has the right
        # amount of arguments, as Python will do this
        # automatically (also with a TypeError).
        functools.wraps(validate_function)
        def decorator_wrapper(*function_args, **function_args_dict):
            if len(accepted_arg_types) is not len(accepted_arg_types):
                raise InvalidArgumentNumberError(validate_function.__name__)
 
            # We're using enumerate to get the index, so we can pass the
            # argument number with the incorrect type to ArgumentValidationError.
            for arg_num, (actual_arg, accepted_arg_type) in enumerate(zip(function_args, accepted_arg_types)):
                if not type(actual_arg) is accepted_arg_type:
                    print(actual_arg, accepted_arg_types)
                    ord_num = ordinal(arg_num + 1)
                    raise ArgumentValidationError(ord_num,
                                                  validate_function.__name__,
                                                  accepted_arg_type)
 
            return validate_function(*function_args)
        return decorator_wrapper
    return accept_decorator

In [ ]:
def returns(*accepted_return_type_tuple):
    '''
    Validates the return type. Since there's only ever one
    return type, this makes life simpler. Along with the
    accepts() decorator, this also only does a check for
    the top argument. For example you couldn't check
    (<type 'tuple'>, <type 'int'>, <type 'str'>).
    In that case you could only check if it was a tuple.
    '''
    def return_decorator(validate_function):
        # No return type has been specified.
        if len(accepted_return_type_tuple) == 0:
            raise TypeError('You must specify a return type.')
 
    	functools.wraps(validate_function)
        def decorator_wrapper(*function_args):
            # More than one return type has been specified.
            if len(accepted_return_type_tuple) > 1:
                raise TypeError('You must specify one return type.')
 
            # Since the decorator receives a tuple of arguments
            # and the is only ever one object returned, we'll just
            # grab the first parameter.
            accepted_return_type = accepted_return_type_tuple[0]
 
            # We'll execute the function, and
            # take a look at the return type.
            return_value = validate_function(*function_args)
            return_value_type = type(return_value)
 
            if return_value_type is not accepted_return_type:
                raise InvalidReturnType(return_value_type,
                                        validate_function.__name__) 
 
            return return_value
 
        return decorator_wrapper
    return return_decorator

`ImageGenerator` is a convenience class that provides management and support for
generating data set images.

In [ ]:
class ImageGenerator(object): 
    """A custom convenience class to support generating image datasets for use with Pillow"""
    def __init__(self, text='A', klass=0, image_id=0, image_props=None, image=None, output_dir='data', filename='UNAMED.png'):
        self._text = text
        self._klass = klass
        self._image_id = image_id
        self._image = image
        self._image_props = image_props
        self._output_dir = output_dir
        self._filename = 'UNAMED.png'
        self._saved = False
        
    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    @property
    def text(self):
        """The text to be rendered on the image. Probably should be a letter"""
        return self._text
    
    @property
    def filename(self):
        """The name of the file. This will initialize 
        a new ImageProps object to use its default properties, 
        and provide a default file name `original` suffix.
        """
        if not self._image_props:
            self._image_props = ImageProps()

        self._filename = "%s-%s-%s" % (self._klass, self._image_id, self._image_props.name)
        if (self._filename == "0-0-original"):
            print("This is the first image in the series! [0-0-original]")
        return self._filename
    
    @property
    def output_dir(self):
        """The output directory where the generated images
        will be stored. Used in the saving and naming of the file.
        """
        return self._output_dir
    
    @property
    def klass(self):
        """Class identifier used to map to a specific character.
        'A' should always be 0. Used in the naming of the file."""
        return self._klass
    
    @property
    def image_props(self):
        """Image id - should be incremented for each image generated. Used
        in the naming of the file."""
        return self._image_props
    
    @image_props.setter
    def image_props(self, value):
        """ImageProps object.  This object must be of type ImageProps"""
        if not isinstance(value, ImageProps):
            raise TypeError("'image_props' must be a type of ImageProps")
        
        if (value == self._image_props):
            raise ValueError("Incoming image_props is identical to current! Are you sure you want that to happen?")
        
        if self._saved:
            self._saved = False
            
        self._image_props = value

    @property
    def image_id(self):
        """Image id - should be incremented for each image generated. Used
        in the naming of the file."""
        return self._image_id
    
    @property
    def image(self):
        """Returns the test object image.  `image` is 
        of type Pil.Image"""
        img = Image.new()
        return self._image
    
    @image.setter
    def image(self, value):
        """Sets the image to be used.  This should normally be set
        during initialization, but this is here for convenience. The
        value must be an instance of PIL.Image."""
        if not isinstance(value, Image):
            raise TypeError("Image must be of type PIL.Image")
        self._image = value
        
    @property
    def saved(self):
        """Convenience value to prevent images being saved twice."""
        return self._saved
    
    @accepts(list, list)
    def process_characters(characters=[], fonts=[FontProps]):
        """Takes a list of characters, generates the data set"""
        #
        # Cycle through fonts
        #
        print(characters)
        fonts = fonts
        for font in fonts:
            #
            # Cycle through upper case characters
            # and set the cls to 0
            #
            cls = 0
            for val in test_characters:
                try:
                    # Generate initial image - no rotation
                    orig = generate_image(str(val), unicode(str(font)), cls, idx, 'orig', 0, 'data', '#FFFFFF')
                    # Generate 7 images
                    deg = 45;
                    for i in range(7):
                        generateRotatedImage(orig, cls, idx, deg)
                        deg = 45 + (45 * (i + 1))
                
                    generateFilteredGuassianBlurImage(orig, cls, idx)
                    generateEnhancedColorImage(orig, cls, idx)
                    generateEhancedContrastedImage(orig, cls, idx)


#             generate_image(str(val), unicode(str(font)), cls, idx, 'dark', 270, 'data','#000000')
                    cls += 1
                    idx += 1
                except: 
                    print "Unexpected error:", sys.exc_info()[0]
                    raise SystemError(sys.exc_info()[0])
                
    def save(self):
        """Saves the test data object.  This will call the filename property
        in order to grab the class, image id, and image_props object.  Since
        this calls `filename`, an instance of the default image_props object
        is created if there isn't one"""
        if self._image and not self._saved:
            self._image.save(unicode(self._filename + ".png"))
            self._saved = True
            print("SAVED IMAGE")
        elif self._saved:
            print("Already saved this image - overwriting not implemented")
        else:
            raise IOError("Unable to save image file - it doesn't exist!  Add a PIL Image object")

    def __str__(self):
        attrs = vars(self)
        desc = '\n'.join("%s: %s" % item for item in attrs.items())
        return desc
        

## Helper Function

`find_coeffs` is a helpful method to allow us to find the 4 points
of a plane in order to perform transformations.

In [ ]:
# NOT USED - but could be used to do pixel level manipulation
# useful for doing perspective mapping.
def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = numpy.matrix(matrix, dtype=numpy.float)
    B = numpy.array(pb).reshape(8)

    res = numpy.dot(numpy.linalg.inv(A.T * A) * A.T, B)
    return numpy.array(res).reshape(8)


`create_letter` takes in a number of arguments and generates
an image with centered text based on its input described here:
* `props` - ImageProps object
* `text` - text to write to imaage (character)

In [ ]:
@accepts(str, ImageProps)
@returns(Image.Image)
def create_image(text='A', props=None):
    """Create an image.  Takes an instance of `ImageProps` and returns an image.
    This method does not save the image, but simply generates it for pipeline
    management."""
    if not props or not isinstance(props, ImageProps):
        raise TypeError("Props cannot be null and must be of type `ImageProps`")
    
    font_file = props.font
    font_size = props.font_size
    font_color = props.text_color
    image_size = props.size
    background_color = props.background_color
    
    #font  =  ImageFont.truetype ( font_path + font_name + ".ttf", font_size )
    font  =  ImageFont.truetype ( font_file, font_size )
    # Create the image with size and background options
    im  =  Image.new ( "RGBA", image_size, background_color )
    # Creat drawing context
    draw  =  ImageDraw.Draw ( im )
    # Attempt to center the font inside the background canvas
    text_x, text_y = font.getsize(text)
    width, height = image_size
    x = (width - text_x)/2
    y = (height - text_y)/2
    
    # Write the text ont h
    draw.text ( (x,y), text, font=font, fill=font_color )
    return im

`name_image` takes in a number of arguments and generates
an unicode file name of the final image.  This should b run last
* `klass` - classification number that correspondes to class used in `ocr.csv`
* `image_id` - autogenerated image id 
* `transform` - type of modification to image for easy reference
* `folder` - where the files should be stored

In [ ]:
@accepts(int, int, str, str)
@returns(unicode)
def name_image(klass,
               image_id,
               transform,
               folder='.'):
    return unicode("{0}/{1}-{2}-{3}.png".format(folder, klass, image_id, transform))

### Supporting Classes

`FontProps` is a convenience class that provides properties to use in transforming images objects. 
It should be injected into the `ImageGenerator` or an `ImageProps` object.

In [ ]:
class FontProps(object):
    """A convenience class that provides properties to use modifying fonts for images objects"""
    def __init__(self,
                 color=(255,255,255,255),
                 size=27,
                 font_file=unicode("/System/Library/Fonts/HelveticaNeue.ttc"),
                ):
        self._color = color
        self._size = size
        self._font_file = font_file

 
    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    @property
    def color(self):
        return self._color
    
    @property
    def size(self):
        return self._size
    
    @property
    def font_file(self):
        return self._font_file
    
    def __str__(self):
        attrs = vars(self)
        return '\n'.join("%s: %s" % item for item in attrs.items())
    

`ImageProps` is a convenience class that provides properties to use in transforming images objects. 
It should be injected into the `ImageGenerator` object.

In [ ]:
class ImageProps(object):
    """A convenience class that provides properties to use in transforming images objects"""
    def __init__(self,
                 name='original',
                 font_props=FontProps(),
                 rotation=0,
                 background_color=(255,255,255,255),
                 contrast=1.0,
                 brightness=1.0,
                 color_adjust=1.0,
                 sharpness=1.0,
                 size=(28, 28),
                ):
        self._name = name
        self._rotation = rotation
        self._background_color = background_color
        self._contrast = contrast
        self._color_adjust = color_adjust
        self._sharpness = sharpness
        self._brightness = brightness
        self._size = size

 
    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    @property
    def name(self):
        return self._name
    
    @property
    def rotation(self):
        return self._rotation
    
#     @rotation.setter
#     def rotation(self, value):
#         self._rotation = value
    
    @property
    def font(self):
        return self._font_props
    
    @property
    def text_color(self):
        return self._font_props.color
    
#     @text_color.setter
#     def text_color(self, value):
#         self._text_color = value
    
    @property
    def background_color(self):
        return self._background_color
    
#     @background_color.setter
#     def background_color(self, value):
#         self._background_color = value
    
    @property
    def contrast(self):
        return self._contrast
    
#     @contrast.setter
#     def contrast(self, value):
#         self._contrast = value
    
    @property
    def sharpness(self):
        return self._sharpness
    
#     @sharpness.setter
#     def sharpness(self, value):
#         self._sharpness = value
    
    @property
    def brightness(self):
        return self._brightness
    
#     @brightness.setter
#     def brightness(self, value):
#         self._brightness = value
    
    @property
    def color_adjust(self):
        return self._color_adjust
    
#     @color.setter
#     def color(self, value):
#         return self._color
    
    @property
    def size(self):
        return self._size
    
#     @size.setter
#     def size(self, value):
#         return self._size
    
    def __str__(self):
        attrs = vars(self)
        return '\n'.join("%s: %s" % item for item in attrs.items())
    


**&nbsp; &nbsp; 1. First we create a Font Property object and an Image Property object**

This will default to a black text, white background, no rotation, and
a file suffix of `original`. 

In [ ]:
default_font_props = FontProps()
default_image_props = ImageProps()
print(default_image_props)

Lets test to see if we can set any props:

**`Note:` `name` should be immutable!**

In [ ]:
# default_image_props.name = 'bob' #  AttributeError can't set attribute
# default_image_props.color = 'black' # AttributeError can't set attribut
default_image_props.name

We have to initialize the object with our values.  This will encourage
a declaritve approach to our image processing, while providing some
immutability to ensure we are getting the props we desire.

In [ ]:
myprops = ImageProps('rotate_90',
                     background_color=(185,255,185,255),
                     rotation=30)

Here are the **NEW** attributes *after* modifications:

In [ ]:
print(myprops)

**&nbsp; &nbsp; 2. Then we create our Test Data object**

We initialize it with our `image_props` object.

In [ ]:
data = ImageGenerator(text='C', klass=1, image_id=2, image_props=myprops)
# data = ImageGenerator(image_props=default_image_props) # Error! cannot add same image_props to ImageGenerator

In [ ]:
print(data)

Note that when we call the `filename property` it updates it.  

In [ ]:
print(data.filename)

In [ ]:
print(data.saved)

The following will error out due to the fact that we don't 
allow a `ImageGenerator object` to be initialized with the same image_props.

In [ ]:
# data.image_props = default_image_props

Now our `filename` property in our `ImageGenerator` object *data* should be updated as well:  

In [ ]:
print(data)

Lets try to save some data:

In [ ]:
print(dir(ImageProps))


In [ ]:
# data.save() # IOError! No image

We can't save the file because we haven't added an image yet. The
following error will show up:

```IOError: Unable to save image file - it doesn't exist!  Add a PIL Image object```

**&nbsp; &nbsp; 3. We now want to set some properties**

Lets initialize our test_data with a default image property object.
This will give us no tranformation objects. Here is a look at the 
default properties:

In [ ]:
print(myprops)

In [ ]:
font_props = FontProps()
print(font_props)

In [ ]:


test_props = ImageProps('background',
                        background_color=(12,145,234,255),
                        font_props=font_props,
                        contrast=0.25
                       )

test_data = ImageGenerator(image_props=test_props)
print(test_data)

### Supported Characters

Currently the character set supported is `[a-z, A-Z, 0-9, '\'', '\,']`

In [ ]:
uppercase=[ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
lowercase=[ 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z' ]
numbers=[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
characters=[ "'", "," ]
test_characters = uppercase + lowercase + numbers + characters

### CSV Setup

This is used to map classification of the images to the actual image.  We create the file `ocr.csv` and make it writeable:

In [ ]:
myfile = open('ocr.csv','w') 
myfile.write("Class")

### Load fonts

We want to check and see what fonts are available:

In [ ]:
fonts = fontconfig.query( lang='en' )

In [ ]:
### Check if font has character

## Clean up fonts

The following fonts seemed to break my proces due to the unicode character
sets.  

In [ ]:
try:
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W0.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ明朝 ProN.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ明朝 ProN.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W1.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W2.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W5.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W6.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W6.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W7.ttc") 
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W8.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W8.ttc")
    fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W9.ttc")
except ValueError:
    pass # or scream: thing not in some_list!
except AttributeError:
    print "Unexpected error:", sys.exc_info()[0]
    pass

#### Available fonts

The following number is how many fonts are available to be processed:

In [ ]:
len(fonts)

#### Display fonts

In [ ]:
#for font in fonts:
 #   print(font)

## Test image

##### Setup

We create a 28x28 image and also chose colors and font:
* `width` - width of image
* `height` - height of image
* `font_size` - size of font
* `image_size` - tuple, we want a 28x28 pixel image to process
* `background_color` - color behind text
* `text_color` - color of text
* `font_file` - location of font
* `transform` - type of transform - defaults to 'orig'

In [ ]:
image_props = ImageProps(font_props)

print(image_props)
transform = 'normal'
folder = 'data'

##### Sample Character and initialize indexes for naming conventions

In [ ]:
cls = 0
idx = 0
transform = 'orig'

## Generate Images

##### Helper function

In [ ]:
@accepts(ImageProps, int, str, str)
def generate_image(image_props=None):
    if not folder:
        folder='data'
    #
    # Create image
    #
    orig_img = create_image(
        font_file,
        font_size,
        text_color,
        image_size,
        background,
        text
    )
    rotate = int(rotate)
    if rotate != 0:
        orig_img = orig_img.rotate(rotate)
    #
    # Save Image
    #
    orig_img.save(name_image(klass, image_id, transform, folder))
    return orig_img
    

*Loop through fonts*

**Uncomment - for testing only!!!**

In [ ]:
test_characters = ['A']

### Filters

Filters allow us to alter the image to create more test sets:

`generateFilteredBlureImages` - Generates an image with Gaussian Blur with specified radius:

In [ ]:
def generateFilteredGuassianBlurImage(image, cls, idx):
    radius = 1
    filter = ImageFilter.GaussianBlur(radius)
    img = image.filter(filter)
    img.save(name_image(cls, idx, 'g-blur', 'data'))

`generateFilteredBoxBlurImage` - Generates an image with Box Blur and specified radius:

In [ ]:
## only available in Pillow 4.3
def generateFilteredBoxBlurImage(image, cls, idx):
    radius = 5
    filter = ImageFilter.BoxBlur(radius)
    img = image.filter(filter)
    img.save(name_image(cls, idx, 'box-blur', 'data'))

`generateEnhancedColorImages` - Generates an image with varying color level:

In [ ]:
# not useful on black and white images
def generateEnhancedColorImage(image, cls, idx, factor):
    enhancer = ImageEnhance.Color(image)
    factor = 0.25
    img = enhancer.enhance(factor)
    img.save(name_image(cls, idx, 'color', 'data'))

`generateEhancedContrastedImage` - Generates an image with varying contrast level:

In [ ]:
def generateEhancedContrastedImage(image, cls, idx, factor):
    enhancer = ImageEnhance.Contrast(image)
    # (0.0...1.0)
    factor = 0.
    img = enhancer.enhance(factor)
    img.save(name_image(cls, idx, 'contrast', 'data'))
        

`generateRotatedImages` - generate rotated images from an image:

In [ ]:
def generateRotatedImage(image, cls, idx, degrees=0):
    img = image.rotate(degrees); 
    img.save(name_image(cls, idx, 'rotate_' + str(degrees), 'data'))

## MAIN - Process Images

In [ ]:
#
# Cycle through fonts
#
print(test_characters)
for font in fonts:
    #
    # Cycle through upper case characters
    # and set the cls to 0
    #
    cls = 0
    for val in test_characters:
        try:
            # Generate initial image - no rotation
            orig = generate_image(str(val), unicode(str(font)), cls, idx, 'orig', 0, 'data', '#FFFFFF')
             # Generate 7 images
#             deg = 45;
#             for i in range(7):
#                 generateRotatedImage(orig, cls, idx, deg)
#                 deg = 45 + (45 * (i + 1))
                
#             generateFilteredGuassianBlurImage(orig, cls, idx)
#             generateEnhancedColorImage(orig, cls, idx)
#             generateEhancedContrastedImage(orig, cls, idx)


            #generate_image(str(val), unicode(str(font)), cls, idx, 'dark', 270, 'data','#000000')
            cls += 1
            idx += 1
        except: 
            print "Unexpected error:", sys.exc_info()[0]
            print(font)
            print(val) 
            raise

*Check if character exists in this font*

*If it exists, then generate image*

In [ ]:
import numpy

def generate_letter(contrast_energy = .01, #michelson contrast energy
                   noise = 30,
                   bg_luminance = 128,
                   letter = "a",
                   letter_size = 28):
    N = 28 #size of image in pixels
    #first figure out what is the ink-area of the letter
    font = ImageFont.truetype("/System/Library/Fonts/HelveticaNeue.ttc", letter_size)
    #we copy the .ttf file to the local directory to avoid problems
    im_temp = Image.new("1", (1,1), 0)
    draw = ImageDraw.Draw(im_temp)
    #now we can draw on this

    sz = draw.textsize(letter, font=font)
    # this tells us the size of the letter
    im_temp = Image.new("1", sz, 0)
    #this is a temporary binary image created solely for the purpose of computing
    #the ink-area of the letter
    draw = ImageDraw.Draw(im_temp)
    #now we can draw on this
    draw.text((0,0), letter, font=font, fill=1)
    pix = im_temp.load()
    #pix is now an addressable array of pixel values
    area_in_pixels = 0.
    for row in xrange(sz[0]):
        for col in xrange(sz[1]):
            area_in_pixels += pix[row,col]

    #since contrast_energy = contrast^2 * pixel_area
    contrast = (contrast_energy/area_in_pixels)**0.5
    fg_luminance = bg_luminance*(1+contrast)/(1-contrast)
    print area_in_pixels
    print contrast
    print fg_luminance


    im = Image.new("L", (N,N), bg_luminance)
    #im is now a NxN luminance image with luminance set to bg_luminance

    draw = ImageDraw.Draw(im)
    #now we can draw on this

    draw.text(((N-sz[0])/2, (N-sz[1])/2), letter, font=font, fill=1)
    #this centers the letter

    if noise > 0:
        pix = im.load()
    #pix is now an addressable array of pixel values

    rd = numpy.random.normal(scale=noise, size=(N,N))
    print(rd)
    for row in xrange(N):
        for col in xrange(N):
            print(col)
            pix[row,col] += rd[row,col)]
    

In [ ]:
generate_letter(.01,
                30,
                128,
                "a",
                28)

### Cleanup and exit

### Close file

In [ ]:
myfile.close()

### Read File

In [ ]:
f = open("ocr.csv","r") 

In [ ]:
file_contents = f.read()

### Output of file

In [ ]:
print(file_contents)